In [1]:
import pandas as pd
import os
from typing import List,Dict,Any

In [2]:
from langchain_core.documents import Document
from langchain_text_splitters import(
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    TokenTextSplitter
)
print('Setup Completed')

Setup Completed


### Understanding Document Structure in Langchain

In [3]:
#create a simple Document
doc = Document(
    page_content = 'This is a test document',
    metadata = {'source': 'test.txt',
                'author':'Sandeep Vissa'}
)
print('Printing Document structure')
print(f'content: {doc.page_content}')
print(f'metadata: {doc.metadata}')

Printing Document structure
content: This is a test document
metadata: {'source': 'test.txt', 'author': 'Sandeep Vissa'}


### Reading Text(.txt) Files

In [4]:
#create simple txt files
import os
os.makedirs('data/text_files',exist_ok=True)

In [5]:
sample_texts={
    "data/text_files/python_intro.txt":"""Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",
    
    "data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems
    
    
    """

}

for filepath,content in sample_texts.items():
    with open(filepath,'w',encoding="utf-8") as f:
        f.write(content)

print("✅ Sample text files created!")

✅ Sample text files created!


### Text Loader

In [11]:

from langchain.document_loaders import TextLoader
loader = TextLoader('data/text_files/python_intro.txt',encoding='utf-8')
documents = loader.load()
#print(type(documents))
#print(documents)
print(f'loaded {len(documents)} Document')
print(f'Content preview: {documents[0].page_content[:100]}')
print(f'Metadata: {documents[0].metadata}')

loaded 1 Document
Content preview: Python Programming Introduction

Python is a high-level, interpreted programming language known for 
Metadata: {'source': 'data/text_files/python_intro.txt'}


### Directory Loader - For multiple text files

In [12]:
from langchain.document_loaders import DirectoryLoader

dir_loader = DirectoryLoader(
    'data/text_files',
    glob='**/*.txt',
    show_progress=True,
    loader_cls=TextLoader,
    loader_kwargs={'encoding':'utf-8'}
)

documents = dir_loader.load()
print(f'loaded {len(documents)} Document')

for i,doc in enumerate(documents):
    print(f'\nDocument {i+1}:')
    print(f'Source: {doc.metadata["source"]}')
    print(f'Length: {len(doc.page_content)} characters')

100%|██████████| 2/2 [00:00<00:00, 241.05it/s]

loaded 2 Document

Document 1:
Source: data/text_files/python_intro.txt
Length: 489 characters

Document 2:
Source: data/text_files/machine_learning.txt
Length: 583 characters


### Text Splitters

In [13]:
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    TokenTextSplitter
)

print(documents)

[Document(metadata={'source': 'data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.'), Document(metadata={'source': 'data/text_files/machine_learning.txt'}, page_content='Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervised Learn

In [14]:
text = documents[0].page_content
print(text)

Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.


In [17]:
char_splitter = CharacterTextSplitter(
    separator = '\n',
    chunk_size = 200,
    chunk_overlap = 20,
    length_function = len
)

char_chunks = char_splitter.split_text(text)
print(f'Created {len(char_chunks)} chunks')
print(f'First chunk: {char_chunks[0][:100]}')

Created 3 chunks
First chunk: Python Programming Introduction
Python is a high-level, interpreted programming language known for i


In [18]:
recursive_splitter = RecursiveCharacterTextSplitter(
    separators=['\n'],
    chunk_size=200,
    chunk_overlap=20,
    length_function=len
)

rec_chunks = recursive_splitter.split_text(text)
print(f'Created {len(rec_chunks)} chunks')
print(f'First chunk: {rec_chunks[0][:100]}')

Created 3 chunks
First chunk: Python Programming Introduction

Python is a high-level, interpreted programming language known for 


In [19]:
# Create text without natural break points
simple_text = "This is sentence one and it is quite long. This is sentence two and it is also quite long. This is sentence three which is even longer than the others. This is sentence four. This is sentence five. This is sentence six."

splitter = RecursiveCharacterTextSplitter(
    separators=[" "],  # Only split on spaces
    chunk_size=80,
    chunk_overlap=20,
    length_function=len
)

chunks = splitter.split_text(simple_text)

print(f"\nSimple text example - {len(chunks)} chunks:\n")

for i in range(len(chunks) - 1):
    print(f"Chunk {i+1}: '{chunks[i]}'")
    print(f"Chunk {i+2}: '{chunks[i+1]}'")
    
    
    print()


Simple text example - 4 chunks:

Chunk 1: 'This is sentence one and it is quite long. This is sentence two and it is also'
Chunk 2: 'two and it is also quite long. This is sentence three which is even longer than'

Chunk 2: 'two and it is also quite long. This is sentence three which is even longer than'
Chunk 3: 'is even longer than the others. This is sentence four. This is sentence five.'

Chunk 3: 'is even longer than the others. This is sentence four. This is sentence five.'
Chunk 4: 'is sentence five. This is sentence six.'

